# 📰 News Recommendation System Part 4 - Ranking

## 📘 Project Introduction
This project explores user behavior prediction in a news recommendation scenario. The goal is to build a model that can predict a user's future click behavior based on their historical browsing and clicking behavior data, specifically the last news article they clicked on.

The setting is inspired by a real-world news app, where delivering timely, relevant content is essential for user engagement. This project aims to simulate a practical recommender system, combining business intuition with machine learning techniques to address a realistic problem in the content recommendation space.

## 📊 Data Overview
The dataset contains user interaction data from a large-scale news platform, including:
- 300,000 users
- ~3 million clicks
- 360,000+ unique news articles; each news article is represented by a pre-trained embedding vector, capturing semantic relationships between articles.

We extracted click log data from 200,000 users as the training set, 50,000 users as test set A, and 50,000 users as test set B.

## 📄 Data Tables

- train_click_log.csv: Training set user click logs
- testA_click_log.csv: Test set user click logs
- articles.csv: News article information data table
- articles_emb.csv: Embedding vector representation of news articles

|        **Field**        |         **Description**          |
| :---------------------: | :------------------------------: |
|         user_id         |              User ID             |
|    click_article_id     |            Clicked article ID    |
|     click_timestamp     |            Click timestamp        |
|    click_environment    |             Click environment     |
|    click_deviceGroup    |            Click device group     |
|        click_os         |           Click operating system  |
|      click_country      |             Click city            |
|      click_region       |             Click region          |
|   click_referrer_type   |           Click source type       |
|       article_id        | Article ID, corresponding to click_article_id |
|       category_id       |            Article type ID        |
|      created_at_ts      |          Article creation timestamp |
|       words_count       |             Article word count     |
| emb_1,emb_2,...,emb_249 |      Article embedding vector representation |

## 📏 Evaluation Metrics
The final recommendation for each user will include five recommended articles, sorted by click probability.

For example, for user1, our recommendation would be:
> user1, article1, article2, article3, article4, article5.

There is only one correct answer for each user's last clicked article, so we check if any of the recommended five articles match the actual answer. We will use **mean reciprocal rank** as the evaluation metric. The formula is as follows:
$$
score(user) = \sum_{k=1}^5 \frac{s(user, k)}{k}
$$

If article1 is the actual article clicked by the user, then s(user1, 1) = 1, and s(user1, 2-4) are all 0. If article2 is the article clicked by the user, then s(user, 2) = 1/2, and s(user, 1, 3, 4, 5) are all 0. Thus, score(user) = the reciprocal of the rank at which the match occurs. If there are no matches, score(user1) = 0. This is reasonable because we want hits to be as high-ranking as possible, which yields a higher score.

## 💡 Project Understanding
The goal of this project is to **predict the last news article a user clicked, based on their historical browsing data**. Unlike traditional structured prediction problems, this is more aligned with real-world recommendation systems, using raw user click logs rather than neatly labeled data.

To approach this, I framed the task as a **supervised learning** problem by transforming user-article interactions into "features + labels" training data. The core idea is to predict the likelihood of a user clicking a given article, turning this into a click-through rate (CTR) prediction task. This reframing allows for the use of **classification models**—starting with simple baselines like logistic regression and moving toward deep learning approaches.

Now, we have converted this problem into a classification problem, where the classification label is whether the user will click on a particular article. The features of the classification problem will include the user and the article. We need to train a classification model to predict the probability of a particular user clicking on a specific article. This raises several additional questions:
- How to create training and testing datasets?
- What specific features can we leverage?
- What models can we attempt?
- With 360,000 articles and over 200,000 users, what strategies do we have to reduce the problem's scale? How do we make the final predictions?

**For the fourth part, we will use the features from Part 3 to train machine learning models to rank the recalled candidates and predict users' click through rate.**

## The Ranking Phase
After the recall phase, we have successfully narrowed down a huge list of articles to a smaller, more manageable set of candidates for each user. Now, the goal is to use a machine learning model to rank these candidates and find the ones a user is most likely to click on.

We'll use the rich features we've engineered—including user attributes, article attributes, and features derived from the relationship between the user and the articles—to predict a click probability for each candidate. The top-k articles with the highest predicted probabilities will be our final recommendation.

### The Models
We'll use three popular and powerful models for this ranking task:

1. LGBM Ranking Model: A LightGBM model configured specifically for ranking.

2. LGBM Classification Model: A LightGBM model configured for binary classification (to predict a click or no-click).

3. DIN (Deep Interest Network): A deep learning classification model that is highly effective for e-commerce and recommendation tasks.

### Model Ensembling
After we get the output from these models, we won't just pick one. We'll use two classic model ensembling techniques to combine their strengths for even better performance:

1. Weighted Fusion: We'll combine the output scores from each model using a weighted average. This lets us give more importance to the models that perform better.

2. Stacking: We'll use a second, simpler model to take the outputs of our three ranking models and use them as its input features. This allows a meta-model to learn how to best combine the predictions of the base models.

## 1. Import Packages

In [1]:
import numpy as np
import pandas as pd
import pickle
from tqdm import tqdm
import gc, os
import time
from datetime import datetime
import lightgbm as lgb
from sklearn.preprocessing import MinMaxScaler
import warnings
warnings.filterwarnings('ignore')

# Import deepctr
from deepctr.models import DIN
from deepctr.feature_column import SparseFeat, VarLenSparseFeat, DenseFeat, get_feature_names
from tensorflow.keras.preprocessing.sequence import pad_sequences

from tensorflow.keras import backend as K
from tensorflow.keras.layers import *
from tensorflow.keras.models import *
from tensorflow.keras.callbacks import *
import tensorflow as tf

import os
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "2"

ModuleNotFoundError: No module named 'lightgbm'

## 2. Read Features for the Ranking Models

In [ ]:
# If using Google Colab, use this cell to load data
from google.colab import drive

# Connect to Google Drive
drive.mount('/content/drive')

# Define file paths
data_path = '/content/drive/MyDrive/Datasets/news-rec-sys/'
save_path = '/content/drive/MyDrive/Datasets/news-rec-sys/temp_results/'

In [ ]:
# If using a local machine
data_path = './data/' 
save_path = './data/temp_results/' # save temperary result

In [ ]:
offline = False

# When rereading the data, we found that 'click_article_id' is a float, so we convert it to an integer type.
trn_user_item_feats_df = pd.read_csv(save_path + 'trn_user_item_feats_df.csv')
trn_user_item_feats_df['click_article_id'] = trn_user_item_feats_df['click_article_id'].astype(int)

if offline:
    val_user_item_feats_df = pd.read_csv(save_path + 'val_user_item_feats_df.csv')
    val_user_item_feats_df['click_article_id'] = val_user_item_feats_df['click_article_id'].astype(int)
else:
    val_user_item_feats_df = None

tst_user_item_feats_df = pd.read_csv(save_path + 'tst_user_item_feats_df.csv')
tst_user_item_feats_df['click_article_id'] = tst_user_item_feats_df['click_article_id'].astype(int)

# For convenience during feature engineering, we also give the test set an invalid label. We can simply delete it now.
del tst_user_item_feats_df['label']

## 3. Save Ranking Results

In [ ]:
def save_results(recall_df, topk=5, model_name=None):
    # Sort the DataFrame by 'user_id' and 'pred_score'.
    recall_df = recall_df.sort_values(by=['user_id', 'pred_score'])

    # Add a 'rank' column to show the predicted rank for each user.
    recall_df['rank'] = recall_df.groupby(['user_id'])['pred_score'].rank(ascending=False, method='first')

    # Check that each user has at least 'topk' articles.
    tmp = recall_df.groupby('user_id').apply(lambda x: x['rank'].max())
    assert tmp.min() >= topk

    # Delete the 'pred_score' column.
    del recall_df['pred_score']

    # Select the top 'topk' predictions for each user and reshape the DataFrame.
    results = recall_df[recall_df['rank'] <= topk].set_index(['user_id', 'rank']).unstack(-1).reset_index()

    # Rename the column names (return 5 recommended articles).
    results.columns = [int(col) if isinstance(col, int) else col for col in results.columns.droplevel(0)]
    results = results.rename(columns={'': 'user_id', 1: 'article_1', 2: 'article_2',
                                    3: 'article_3', 4: 'article_4', 5: 'article_5'})

    # Generate a filename for the CSV file.
    save_name = save_path + model_name + '_' + datetime.today().strftime('%m-%d') + '.csv'

    # Save the DataFrame to a CSV file without the index and with a header.
    results.to_csv(save_name, index=False, header=True)

In [ ]:
def norm_sim(sim_df, weight=0.0):
    min_sim = sim_df.min()  # Calculate the minimum similarity value.
    max_sim = sim_df.max()  # Calculate the maximum similarity value.

    if max_sim == min_sim:
        sim_df = sim_df.apply(lambda sim: 1.0)  # If max and min are equal, normalize all similarities to 1.0.
    else:
        sim_df = sim_df.apply(lambda sim: 1.0 * (sim - min_sim) / (max_sim - min_sim))  # 对相似度进行归一化

    sim_df = sim_df.apply(lambda sim: sim + weight)  # Add a weight to the normalized similarity.

    return sim_df

## 4. LightGBM Ranking Model
The code in this section is designed to prevent data loss if an error occurs. It ensures that the original DataFrames remain unchanged by creating a copy for the ranking model's use.

The code performs the following actions:

- It copies the training set feature data (trn_user_item_feats_df) to a new DataFrame called trn_user_item_feats_df_rank_model.

- If the script is in offline validation mode, it also copies the validation set feature data (val_user_item_feats_df) to a new DataFrame called val_user_item_feats_df_rank_model.

- It copies the test set feature data (tst_user_item_feats_df) to a new DataFrame called tst_user_item_feats_df_rank_model.

In [ ]:
# Prevent data reloading if an error occurs in the middle of the process.
trn_user_item_feats_df_rank_model = trn_user_item_feats_df.copy()

if offline:
    val_user_item_feats_df_rank_model = val_user_item_feats_df.copy()

tst_user_item_feats_df_rank_model = tst_user_item_feats_df.copy()

In [ ]:
# Define Feature Columns
lgb_cols = ['sim0', 'time_diff0', 'word_diff0','sim_max', 'sim_min', 'sim_sum',
            'sim_mean', 'score','click_size', 'click_time_diff_mean', 'active_level',
            'click_environment','click_deviceGroup', 'click_os', 'click_country',
            'click_region','click_referrer_type', 'user_time_hob1', 'user_time_hob2',
            'words_hbo', 'category_id', 'created_at_ts','words_count', 'user_num', 
            'article_time_diff_mean', 'hot_level']

In [ ]:
# Grouping for the Ranking Model
trn_user_item_feats_df_rank_model.sort_values(by=['user_id'], inplace=True)
g_train = trn_user_item_feats_df_rank_model.groupby(['user_id'], as_index=False).count()["label"].values

if offline:
    val_user_item_feats_df_rank_model.sort_values(by=['user_id'], inplace=True)
    g_val = val_user_item_feats_df_rank_model.groupby(['user_id'], as_index=False).count()["label"].values

In [ ]:
# LightGBM 
lgb_ranker = lgb.LGBMRanker(boosting_type='gbdt', num_leaves=31, reg_alpha=0.0, reg_lambda=1,
                            max_depth=-1, n_estimators=100, subsample=0.7, colsample_bytree=0.7, subsample_freq=1,
                            learning_rate=0.01, min_child_weight=50, random_state=2018, n_jobs= 16)

### LightGBM Ranking Model Parameters
* `boosting_type`: The type of boosting algorithm. We're using 'gbdt', which stands for Gradient Boosting Decision Tree.
* `num_leaves`: The maximum number of leaves in one tree. Setting this to 31 is a common default.
* `reg_alpha (L1 Regularization)`: A coefficient for L1 regularization. We're setting it to 0.0, which means L1 regularization is not being used to control model complexity.
* `reg_lambda (L2 Regularization)`: A coefficient for L2 regularization. We're setting it to 1, which means we are using L2 regularization.
* `max_depth`: The maximum depth of each tree. We've set it to -1, which means there is no limit on the depth.
* `n_estimators`: The number of trees in the model. We're training a total of 100 trees.
* `subsample`: The fraction of the training data to be randomly sampled for each tree. We're using 0.7, or 70%.
* `colsample_bytree`: The fraction of features (columns) to be randomly sampled for each tree. We're also using 0.7.
* `subsample_freq`: The frequency at which subsampling occurs. Setting it to 1 means we'll use a new random subsample for every tree.
* `learning_rate`: Controls the step size of each iteration. A smaller value like 0.01 means the model takes more gradual steps, which can improve accuracy.
* `min_child_weight`: This is a pruning parameter that controls the minimum sum of weights of all data points in a child leaf. We've set it to 50 to help prevent the model from overfitting.
* `random_state`: The random seed. Setting this to 2018 ensures that our results are reproducible.
* `n_jobs`: The number of threads to use for training. We're using 16 threads to speed up the process.

In [ ]:
# Train the lighBGM Ranking model
if offline:
    lgb_ranker.fit(trn_user_item_feats_df_rank_model[lgb_cols], trn_user_item_feats_df_rank_model['label'], group=g_train,
                eval_set=[(val_user_item_feats_df_rank_model[lgb_cols], val_user_item_feats_df_rank_model['label'])],
                eval_group= [g_val], eval_at=[1, 2, 3, 4, 5], eval_metric=['ndcg', ], early_stopping_rounds=50, )
else:
    lgb_ranker.fit(trn_user_item_feats_df[lgb_cols], trn_user_item_feats_df['label'], group=g_train)

In [ ]:
# Model prediction
tst_user_item_feats_df['pred_score'] = lgb_ranker.predict(tst_user_item_feats_df[lgb_cols], num_iteration=lgb_ranker.best_iteration_)

# Save these ranking results for later model ensembling
tst_user_item_feats_df[['user_id', 'click_article_id', 'pred_score']].to_csv(save_path + 'lgb_ranker_score.csv', index=False)

In [ ]:
# Re-rank the prediction results and save the results.
rank_results = tst_user_item_feats_df[['user_id', 'click_article_id', 'pred_score']]
rank_results['click_article_id'] = rank_results['click_article_id'].astype(int)
save_results(rank_results, topk=5, model_name='lgb_ranker')

### Evaluation Metric - NDCG
NDCG stands for Normalized Discounted Cumulative Gain. It's a widely used metric for evaluating the performance of ranking models, especially in information retrieval and recommendation systems. NDCG is effective because it considers two key factors:

1. The relevance of the recommended items.

2. The position of those items in the ranked list.

#### How NDCG Works
When a recommendation system presents you with a list of articles, you are most likely to click on the ones at the very top. An ideal system would place the most relevant articles at the top and less relevant ones further down.

NDCG measures this by:

- Assigning a score to each item: Each recommended item is given a relevance score.

- Discounting by position: The score of each item is "discounted" (given a lower weight) the further down it is on the list. This means a highly relevant item at the top of the list contributes much more to the final score than a highly relevant item at the bottom.

- Normalizing: The final score is normalized to a value between 0 and 1 by dividing it by the score of an "ideal" ranking, where all the most relevant items are placed at the very top.

A higher NDCG score (closer to 1) indicates that the model is doing a better job of putting the most relevant items in the highest-ranked positions, which leads to a better user experience and higher recommendation quality.

### K-fold Cross-validation.

In [ ]:
# 5-fold cross-validation. The 5-fold split is performed by user.
# This section is separate from the single training and validation from before.
def get_kfold_users(trn_df, n=5):
    user_ids = trn_df['user_id'].unique()
    user_set = [user_ids[i::n] for i in range(n)]
    return user_set

k_fold = 5
trn_df = trn_user_item_feats_df_rank_model
user_set = get_kfold_users(trn_df, n=k_fold)

score_list = []
score_df = trn_df[['user_id', 'click_article_id','label']]
sub_preds = np.zeros(tst_user_item_feats_df_rank_model.shape[0])

# Perform 5-fold cross-validation and save the intermediate results for stacking.
for n_fold, valid_user in enumerate(user_set):
    train_idx = trn_df[~trn_df['user_id'].isin(valid_user)] # add slide user
    valid_idx = trn_df[trn_df['user_id'].isin(valid_user)]

    # Group the training and validation sets by user
    train_idx.sort_values(by=['user_id'], inplace=True)
    g_train = train_idx.groupby(['user_id'], as_index=False).count()["label"].values

    valid_idx.sort_values(by=['user_id'], inplace=True)
    g_val = valid_idx.groupby(['user_id'], as_index=False).count()["label"].values

    # Define the model
    lgb_ranker = lgb.LGBMRanker(boosting_type='gbdt', num_leaves=31, reg_alpha=0.0, reg_lambda=1,
                            max_depth=-1, n_estimators=100, subsample=0.7, colsample_bytree=0.7, subsample_freq=1,
                            learning_rate=0.01, min_child_weight=50, random_state=2018, n_jobs= 16)
    # Train the model
    lgb_ranker.fit(train_idx[lgb_cols], train_idx['label'], group=g_train,
                   eval_set=[(valid_idx[lgb_cols], valid_idx['label'])], eval_group= [g_val],
                   eval_at=[1, 2, 3, 4, 5], eval_metric=['ndcg', ], early_stopping_rounds=50, )

    # Predict validation set results
    valid_idx['pred_score'] = lgb_ranker.predict(valid_idx[lgb_cols], num_iteration=lgb_ranker.best_iteration_)

    # Normalize the output scores
    valid_idx['pred_score'] = valid_idx[['pred_score']].transform(lambda x: norm_sim(x))

    valid_idx.sort_values(by=['user_id', 'pred_score'])
    valid_idx['pred_rank'] = valid_idx.groupby(['user_id'])['pred_score'].rank(ascending=False, method='first')

    # Append the validation set prediction results to a list for concatenation later
    score_list.append(valid_idx[['user_id', 'click_article_id', 'pred_score', 'pred_rank']])

    # If in online mode, accumulate the test set predictions from each fold
    if not offline:
        sub_preds += lgb_ranker.predict(tst_user_item_feats_df_rank_model[lgb_cols], lgb_ranker.best_iteration_)

score_df_ = pd.concat(score_list, axis=0)
score_df = score_df.merge(score_df_, how='left', on=['user_id', 'click_article_id'])
# Save the new features generated from the training set cross-validation
score_df[['user_id', 'click_article_id', 'pred_score', 'pred_rank', 'label']].to_csv(save_path + 'trn_lgb_ranker_feats.csv', index=False)

# For the test set, average the predictions from multiple folds. Save the predicted score and rank as new features for stacking.
tst_user_item_feats_df_rank_model['pred_score'] = sub_preds / k_fold
tst_user_item_feats_df_rank_model['pred_score'] = tst_user_item_feats_df_rank_model['pred_score'].transform(lambda x: norm_sim(x))
tst_user_item_feats_df_rank_model.sort_values(by=['user_id', 'pred_score'])
tst_user_item_feats_df_rank_model['pred_rank'] = tst_user_item_feats_df_rank_model.groupby(['user_id'])['pred_score'].rank(ascending=False, method='first')

# Save the new features from the test set cross-validation
tst_user_item_feats_df_rank_model[['user_id', 'click_article_id', 'pred_score', 'pred_rank']].to_csv(save_path + 'tst_lgb_ranker_feats.csv', index=False)

In [ ]:
# Re-rank the prediction results and save the results.
# Generate results for k-fold cross-validated Model.
rank_results = tst_user_item_feats_df_rank_model[['user_id', 'click_article_id', 'pred_score']]
rank_results['click_article_id'] = rank_results['click_article_id'].astype(int)
save_results(rank_results, topk=5, model_name='lgb_ranker')

## 5. LightGBM Classification Model

In [ ]:
# LightGBM Classification Model Definition
lgb_Classfication = lgb.LGBMClassifier(boosting_type='gbdt', num_leaves=31, reg_alpha=0.0, reg_lambda=1,
                            max_depth=-1, n_estimators=500, subsample=0.7, colsample_bytree=0.7, subsample_freq=1,
                            learning_rate=0.01, min_child_weight=50, random_state=2018, n_jobs= 16, verbose=10)

### LightGBM Classifier Parameters

* `boosting_type`: The type of boosting algorithm. We're using `'gbdt'`, which stands for Gradient Boosting Decision Tree.
* `num_leaves`: The maximum number of leaves in each tree. The default is 31. Larger values can increase model complexity and risk overfitting.
* `reg_alpha`: The weight for L1 regularization. The default is `0.0`, which means no L1 regularization is applied to control model complexity and sparsity.
* `reg_lambda`: The weight for L2 regularization. The default is `1.0`, which is used to control model complexity.
* `max_depth`: The maximum depth of each tree. The default is `-1`, which means there is no limit. A larger value can make the model more complex and prone to overfitting.
* `n_estimators`: The number of boosting trees to train. The default is 100.
* `subsample`: The fraction of the training data to be randomly sampled for each tree. The default is `0.7`. This helps prevent overfitting by introducing randomness.
* `colsample_bytree`: The fraction of features (columns) to be randomly sampled for each tree. The default is `0.7`. This also helps prevent overfitting.
* `subsample_freq`: The frequency for subsampling. The default is `1`, meaning a new random sample of data is used for every iteration.
* `learning_rate`: The learning rate, or step size, of the gradient descent. The default is `0.01`. A smaller learning rate can make the model more stable but also slower to train.
* `min_child_weight`: A pruning parameter that defines the minimum sum of weights of all data points required in a child leaf. The default is `50`. This helps control the model's complexity.
* `random_state`: The random seed, set to `2018` for reproducibility.
* `n_jobs`: The number of parallel threads to use for training. The default is `16`.
* `verbose`: Controls the level of output during the training process. The default is `10`, which provides detailed outputs.

In [ ]:
# Training the model
if offline:
    lgb_Classfication.fit(trn_user_item_feats_df_rank_model[lgb_cols], trn_user_item_feats_df_rank_model['label'],
                    eval_set=[(val_user_item_feats_df_rank_model[lgb_cols], val_user_item_feats_df_rank_model['label'])],
                    eval_metric=['auc', ], early_stopping_rounds=50, )
else:
    lgb_Classfication.fit(trn_user_item_feats_df_rank_model[lgb_cols], trn_user_item_feats_df_rank_model['label'])

In [ ]:
# Model Prediction
tst_user_item_feats_df['pred_score'] = lgb_Classfication.predict_proba(tst_user_item_feats_df[lgb_cols])[:,1]

# Save these ranking results for later model ensembling
tst_user_item_feats_df[['user_id', 'click_article_id', 'pred_score']].to_csv(save_path + 'lgb_cls_score.csv', index=False)

In [ ]:
# Re-rank the prediction results and save the results.
rank_results = tst_user_item_feats_df[['user_id', 'click_article_id', 'pred_score']]
rank_results['click_article_id'] = rank_results['click_article_id'].astype(int)
save_results(rank_results, topk=5, model_name='lgb_cls')

In [ ]:
# 5-fold cross-validation. The 5-fold split is performed by user.
# This section is separate from the single training and validation from before.
def get_kfold_users(trn_df, n=5):
    user_ids = trn_df['user_id'].unique()
    user_set = [user_ids[i::n] for i in range(n)]
    return user_set

k_fold = 5
trn_df = trn_user_item_feats_df_rank_model
user_set = get_kfold_users(trn_df, n=k_fold)

score_list = []
score_df = trn_df[['user_id', 'click_article_id', 'label']]
sub_preds = np.zeros(tst_user_item_feats_df_rank_model.shape[0])

# Perform 5-fold cross-validation and save the intermediate results for stacking.
for n_fold, valid_user in enumerate(user_set):
    train_idx = trn_df[~trn_df['user_id'].isin(valid_user)] # add slide user
    valid_idx = trn_df[trn_df['user_id'].isin(valid_user)]

    # Define the model and its parameters
    lgb_Classfication = lgb.LGBMClassifier(boosting_type='gbdt', num_leaves=31, reg_alpha=0.0, reg_lambda=1,
                            max_depth=-1, n_estimators=100, subsample=0.7, colsample_bytree=0.7, subsample_freq=1,
                            learning_rate=0.01, min_child_weight=50, random_state=2018, n_jobs= 16, verbose=10)
    # Train the model
    lgb_Classfication.fit(train_idx[lgb_cols], train_idx['label'],eval_set=[(valid_idx[lgb_cols], valid_idx['label'])],
                          eval_metric=['auc', ], early_stopping_rounds=50, )

    # Predict validation set results
    valid_idx['pred_score'] = lgb_Classfication.predict_proba(valid_idx[lgb_cols],
                                                              num_iteration=lgb_Classfication.best_iteration_)[:,1]

    # Classification models output probabilities directly, so no normalization is needed.
    # valid_idx['pred_score'] = valid_idx[['pred_score']].transform(lambda x: norm_sim(x))

    valid_idx.sort_values(by=['user_id', 'pred_score'])
    valid_idx['pred_rank'] = valid_idx.groupby(['user_id'])['pred_score'].rank(ascending=False, method='first')

    # Append the validation set prediction results to a list for concatenation later
    score_list.append(valid_idx[['user_id', 'click_article_id', 'pred_score', 'pred_rank']])

    # If in online mode, accumulate the test set predictions from each fold
    if not offline:
        sub_preds += lgb_Classfication.predict_proba(tst_user_item_feats_df_rank_model[lgb_cols],
                                                     num_iteration=lgb_Classfication.best_iteration_)[:,1]

score_df_ = pd.concat(score_list, axis=0)
score_df = score_df.merge(score_df_, how='left', on=['user_id', 'click_article_id'])
# Save the new features generated from the training set cross-validation
score_df[['user_id', 'click_article_id', 'pred_score', 'pred_rank', 'label']].to_csv(save_path + 'trn_lgb_cls_feats.csv', index=False)

# For the test set, average the predictions from multiple folds. Save the predicted score and rank as new features for stacking.
tst_user_item_feats_df_rank_model['pred_score'] = sub_preds / k_fold
tst_user_item_feats_df_rank_model['pred_score'] = tst_user_item_feats_df_rank_model['pred_score'].transform(lambda x: norm_sim(x))
tst_user_item_feats_df_rank_model.sort_values(by=['user_id', 'pred_score'])
tst_user_item_feats_df_rank_model['pred_rank'] = tst_user_item_feats_df_rank_model.groupby(['user_id'])['pred_score'].rank(ascending=False, method='first')

# Save the new features from the test set cross-validation
tst_user_item_feats_df_rank_model[['user_id', 'click_article_id', 'pred_score', 'pred_rank']].to_csv(save_path + 'tst_lgb_cls_feats.csv', index=False)

In [ ]:
# Re-rank the prediction results and save the results.
rank_results = tst_user_item_feats_df_rank_model[['user_id', 'click_article_id', 'pred_score']]
rank_results['click_article_id'] = rank_results['click_article_id'].astype(int)
save_results(rank_results, topk=5, model_name='lgb_cls')

## 6. DIN Model

DIN (Deep Interest Network) is a deep learning model for recommendation systems that's designed to better capture a user's changing interests and behavior. Unlike traditional models, it introduces the concept of "interest evolution," recognizing that a user's preferences shift over time based on their past actions.

#### The Core Idea: Attention Mechanism
The central innovation of the DIN model is its use of an attention mechanism to learn how a user's interests evolve. Here's how it works:

1. It takes a user's sequence of historical actions as input.

2. It then looks at a candidate item (the article to be recommended) and calculates an "attention weight" for each of the user's historical actions.

3. These attention weights show which past actions are most relevant to the user's potential interest in the current candidate. For example, if a user recently clicked on several articles about AI, the model would assign a high attention weight to those clicks when considering a new AI article.

4. Finally, it combines the historical actions into a single "interest representation" for the user, which is then used to predict the likelihood of a click.

#### Advantages of DIN
The main strength of the DIN model is its ability to dynamically learn a user's interests. By using an attention mechanism, it can weigh historical actions differently depending on the context of the recommendation. This makes DIN particularly effective for handling sequential behavioral data and adapting to a user's evolving preferences, leading to more accurate and personalized recommendations.

### User Historical Click Behavior List
This is for the DIN model.

In [ ]:
if offline:
    all_data = pd.read_csv(data_path + 'train_click_log.csv')
else:
    trn_data = pd.read_csv(data_path + 'train_click_log.csv')
    tst_data = pd.read_csv(data_path + 'testA_click_log.csv')
    all_data = trn_data.append(tst_data)

In [ ]:
hist_click = all_data[['user_id', 'click_article_id']].groupby('user_id').agg({list}).reset_index()
hist_behavior_df = pd.DataFrame()
hist_behavior_df['user_id'] = hist_click['user_id']
hist_behavior_df['hist_click_article_id'] = hist_click['click_article_id']

In [ ]:
trn_user_item_feats_df_din_model = trn_user_item_feats_df.copy()

if offline:
    val_user_item_feats_df_din_model = val_user_item_feats_df.copy()
else:
    val_user_item_feats_df_din_model = None

tst_user_item_feats_df_din_model = tst_user_item_feats_df.copy()

In [ ]:
trn_user_item_feats_df_din_model = trn_user_item_feats_df_din_model.merge(hist_behavior_df, on='user_id')

if offline:
    val_user_item_feats_df_din_model = val_user_item_feats_df_din_model.merge(hist_behavior_df, on='user_id')
else:
    val_user_item_feats_df_din_model = None

tst_user_item_feats_df_din_model = tst_user_item_feats_df_din_model.merge(hist_behavior_df, on='user_id')

### An Introduction to the DIN Model
DIN (Deep Interest Network) is a deep learning model for recommendation systems that was first introduced by Alibaba in 2018 to address the limitations of earlier models that couldn't capture the diverse interests of users. Its core innovation is an attention mechanism that dynamically represents a user's interests by considering the relevance of their historical behaviors to a specific candidate item.

![](https://deepctr-doc.readthedocs.io/en/v0.8.7/_images/DIN.png)

### How It Works
Instead of creating a single, static vector to represent a user's interests, DIN uses a "local activation unit" that soft-searches a user's historical actions for the parts most relevant to the current candidate. This means:

- High Relevance, High Weight: Past actions that are highly similar to the candidate article receive a higher activation weight, which means they have more influence on the user's final interest score.

- Dynamic Interests: The user's interest vector changes depending on the candidate item. This allows the model to dynamically simulate how a user's interests shift over time, which is perfect for a task like news recommendation where timeliness is key.

### Using the Deepctr Library
We'll use the Deepctr library to implement the DIN model. Here's a brief look at the function signature:

> def DIN(dnn_feature_columns, history_feature_list, dnn_use_bn=False, dnn_hidden_units=(200, 80), ...)

The key parameters are:

* `dnn_feature_columns`: A list of all features.
* `history_feature_list`: A list of features that represent a user's historical behavior.
* `dnn_use_bn`: Controls whether to use Batch Normalization in the deep neural network.
* `dnn_hidden_units`: A list or tuple that defines the number of layers and neurons in the fully connected network.
* `dnn_activation`: The activation function for the fully connected layers (e.g., 'relu').
* `att_hidden_size`: The number of layers and neurons in the fully connected layers of the attention network.
* `att_activation`: The activation function for the attention layers.
* `att_weight_normalization`: A boolean that controls whether to normalize the attention scores.
* `l2_reg_dnn`: The L2 regularization coefficient for the fully connected layers.
* `l2_reg_embedding`: The L2 regularization coefficient for the embedding vectors.
* `dnn_dropout`: The dropout rate for the fully connected layers.
* `task`: The type of task, which can be 'binary' for classification or 'regression'.

### Data Preparation for DIN
Before using the model, we need to preprocess our features into three categories:

1. Sparse Features (Discrete): These are categorical features like user_id. We'll use the SparseFeat function to transform them into low-dimensional embeddings. For this, we will need to specify the column name, the number of unique values in the column, and the desired embedding dimension.

2. Historical Behavior Features: These are sequences of a user's past actions, like the article_ids they've clicked on. We'll use the SparseFeat function first. A key difference in handling these features is that after we get the embedding for each item in a user's history, we need to pass them through an Attention Layer. This layer calculates how relevant each historical action is to the current candidate article. The result is a dynamic user embedding vector that reflects the user's interests based on their past clicks and the specific article being considered. This vector changes with each new candidate, allowing the model to dynamically simulate how user interests evolve over time. Therefore, we need to use VarLenSparseFeat function to pad the sequences so that they all have a uniform length (maxlen). 

3. Dense Features (Continuous): These are numerical features, like word counts or time differences. We'll use the DenseFeat function to define these.

### The Steps for preparing Data for the DIN Model
1. **Initialize Feature Lists**: We start by creating three empty lists to hold our features: sparse_feature_columns for discrete (categorical) features, dense_feature_columns for continuous (numerical) features, and var_feature_columns for historical behavior sequences.

2. **Define Discrete Features**: We iterate through our list of discrete feature names. For each one, we create a SparseFeat object. This object tells the model the column name, the number of unique values (plus one for unknown values), and the dimension of the embedding vector we want to use.

3. **Define Continuous Features**: Similarly, we loop through our numerical feature names and create a DenseFeat object for each. Since these are single numerical values, the dimension is set to 1.

4. **Define Historical Features**: For our user history, we create VarLenSparseFeat objects. These are special because they handle sequences. We first define them as a SparseFeat (giving the column name, vocabulary size, and embedding dimension) and then wrap them in VarLenSparseFeat. This second step is where we specify max_len, which is the maximum sequence length for padding and truncation.

5. **Merge Feature Definitions**: We combine all three lists of feature objects (sparse, dense, and var) into a single list, dnn_feature_columns. This final list serves as the blueprint for our model's input layer.

6. **Process and Prepare Data**: We create an empty dictionary x to hold the actual data. We iterate through the feature names in our blueprint list. If a feature is a historical behavior sequence, we use the pad_sequences function to make sure every sequence has a uniform length of max_len. For all other features, we simply add their values to the x dictionary.

7. **Return the Data**: Finally, the function returns the prepared data dictionary x and the dnn_feature_columns list. This data is now perfectly formatted and ready to be used to train your DIN model.

In [ ]:
# Data preparation function
def get_din_feats_columns(df, dense_fea, sparse_fea, behavior_fea, his_behavior_fea, emb_dim=32, max_len=100):
    """
    Data preparation function.
    df: The dataset.
    dense_fea: A list of continuous (numerical) feature columns.
    sparse_fea: A list of discrete (categorical) feature columns.
    behavior_fea: A list of candidate behavior feature columns.
    his_behavior_fea: A list of user's historical behavior feature columns.
    embedding_dim: The dimension for the embeddings. Here, for simplicity, all discrete features use the same dimension.
    max_len: The maximum length for user sequences.
    """

    # Create SparseFeat objects for the discrete features.
    sparse_feature_columns = [SparseFeat(feat, vocabulary_size=df[feat].nunique() + 1, embedding_dim=emb_dim) for feat in sparse_fea]
    # Create DenseFeat objects for the continuous features.
    dense_feature_columns = [DenseFeat(feat, 1, ) for feat in dense_fea]
    # Create VarLenSparseFeat objects for the historical behavior features.
    var_feature_columns = [VarLenSparseFeat(SparseFeat(feat, vocabulary_size=df['click_article_id'].nunique() + 1,
                                    embedding_dim=emb_dim, embedding_name='click_article_id'), maxlen=max_len) for feat in hist_behavior_fea]
    # Combine all feature objects into a single list.
    dnn_feature_columns = sparse_feature_columns + dense_feature_columns + var_feature_columns

    # Build the input dictionary 'x'.
    x = {}
    for name in get_feature_names(dnn_feature_columns):
        if name in his_behavior_fea:
            # This is a historical behavior sequence.
            his_list = [l for l in df[name]]
            x[name] = pad_sequences(his_list, maxlen=max_len, padding='post')  # 2D array
        else:
            x[name] = df[name].values

    return x, dnn_feature_columns

In [ ]:
# Separate the features
sparse_fea = ['user_id', 'click_article_id', 'category_id', 'click_environment', 'click_deviceGroup',
              'click_os', 'click_country', 'click_region', 'click_referrer_type', 'is_cat_hab']

behavior_fea = ['click_article_id']

hist_behavior_fea = ['hist_click_article_id']

dense_fea = ['sim0', 'time_diff0', 'word_diff0', 'sim_max', 'sim_min', 'sim_sum', 'sim_mean', 'score',
             'rank','click_size','time_diff_mean','active_level','user_time_hob1','user_time_hob2',
             'words_hbo','words_count']

In [ ]:
# Normalize dense features. Neural network training requires that all numerical values are normalized.
mm = MinMaxScaler()

# The following is a special handling process. If there are invalid values (e.g., inf)
# they must be handled before normalization. 
trn_user_item_feats_df_din_model.replace([np.inf, -np.inf], 0, inplace=True)
tst_user_item_feats_df_din_model.replace([np.inf, -np.inf], 0, inplace=True)

for feat in dense_fea:
    trn_user_item_feats_df_din_model[feat] = mm.fit_transform(trn_user_item_feats_df_din_model[[feat]])

    if val_user_item_feats_df_din_model is not None:
        val_user_item_feats_df_din_model[feat] = mm.fit_transform(val_user_item_feats_df_din_model[[feat]])

    tst_user_item_feats_df_din_model[feat] = mm.fit_transform(tst_user_item_feats_df_din_model[[feat]])

In [ ]:
# # Prepare training data
x_trn, dnn_feature_columns = get_din_feats_columns(trn_user_item_feats_df_din_model, dense_fea,
                                               sparse_fea, behavior_fea, hist_behavior_fea, max_len=50)
y_trn = trn_user_item_feats_df_din_model['label'].values

if offline:
    # Prepare validation data
    x_val, dnn_feature_columns = get_din_feats_columns(val_user_item_feats_df_din_model, dense_fea,
                                                   sparse_fea, behavior_fea, hist_behavior_fea, max_len=50)
    y_val = val_user_item_feats_df_din_model['label'].values

dense_fea = [x for x in dense_fea if x != 'label']
x_tst, dnn_feature_columns = get_din_feats_columns(tst_user_item_feats_df_din_model, dense_fea,
                                               sparse_fea, behavior_fea, hist_behavior_fea, max_len=50)

In [ ]:
# Build the model
model = DIN(dnn_feature_columns, behavior_fea)

# View the model structure
model.summary()

# Compile the model
model.compile('adam', 'binary_crossentropy', metrics=['binary_crossentropy', tf.keras.metrics.AUC()])

In [ ]:
# Train the model
if offline:
    history = model.fit(x_trn, y_trn, verbose=1, epochs=10, validation_data=(x_val, y_val) , batch_size=256)
else:
    # history = model.fit(x_trn, y_trn, verbose=1, epochs=3, validation_split=0.3, batch_size=256)
    history = model.fit(x_trn, y_trn, verbose=1, epochs=2, batch_size=256)

In [ ]:
# Model Prediction
tst_user_item_feats_df_din_model['pred_score'] = model.predict(x_tst, verbose=1, batch_size=256)
tst_user_item_feats_df_din_model[['user_id', 'click_article_id', 'pred_score']].to_csv(save_path + 'din_rank_score.csv', index=False)

In [ ]:
# Re-rank the prediction results and save the results.
rank_results = tst_user_item_feats_df_din_model[['user_id', 'click_article_id', 'pred_score']]
save_results(rank_results, topk=5, model_name='din')

In [ ]:
# 5-fold cross-validation. The 5-fold split here is performed by user.
# This section is separate from the single training and validation from before.
def get_kfold_users(trn_df, n=5):
    user_ids = trn_df['user_id'].unique()
    user_set = [user_ids[i::n] for i in range(n)]
    return user_set

k_fold = 5
trn_df = trn_user_item_feats_df_din_model
user_set = get_kfold_users(trn_df, n=k_fold)

score_list = []
score_df = trn_df[['user_id', 'click_article_id', 'label']]
sub_preds = np.zeros(tst_user_item_feats_df_rank_model.shape[0])

dense_fea = [x for x in dense_fea if x != 'label']
x_tst, dnn_feature_columns = get_din_feats_columns(tst_user_item_feats_df_din_model, dense_fea,
                                                   sparse_fea, behavior_fea, hist_behavior_fea, max_len=50)

# Perform 5-fold cross-validation and save the intermediate results for stacking.
for n_fold, valid_user in enumerate(user_set):
    train_idx = trn_df[~trn_df['user_id'].isin(valid_user)] # add slide user
    valid_idx = trn_df[trn_df['user_id'].isin(valid_user)]

    # Prepare training data
    x_trn, dnn_feature_columns = get_din_feats_columns(train_idx, dense_fea,
                                                       sparse_fea, behavior_fea, hist_behavior_fea, max_len=50)
    y_trn = train_idx['label'].values

    # Prepare validation data
    x_val, dnn_feature_columns = get_din_feats_columns(valid_idx, dense_fea,
                                                   sparse_fea, behavior_fea, hist_behavior_fea, max_len=50)
    y_val = valid_idx['label'].values

    # Train the model
    history = model.fit(x_trn, y_trn, verbose=1, epochs=2, validation_data=(x_val, y_val) , batch_size=256)

    # Predict validation set results
    valid_idx['pred_score'] = model.predict(x_val, verbose=1, batch_size=256)

    valid_idx.sort_values(by=['user_id', 'pred_score'])
    valid_idx['pred_rank'] = valid_idx.groupby(['user_id'])['pred_score'].rank(ascending=False, method='first')

    # Append the validation set prediction results to a list for concatenation later
    score_list.append(valid_idx[['user_id', 'click_article_id', 'pred_score', 'pred_rank']])

    # If in online mode, accumulate the test set predictions from each fold
    if not offline:
        sub_preds += model.predict(x_tst, verbose=1, batch_size=256)[:, 0]

score_df_ = pd.concat(score_list, axis=0)
score_df = score_df.merge(score_df_, how='left', on=['user_id', 'click_article_id'])
# Save the new features generated from the training set cross-validation
score_df[['user_id', 'click_article_id', 'pred_score', 'pred_rank', 'label']].to_csv(save_path + 'trn_din_cls_feats.csv', index=False)

# For the test set, average the predictions from multiple folds. Save the predicted score and rank as new features for stacking.
tst_user_item_feats_df_din_model['pred_score'] = sub_preds / k_fold
tst_user_item_feats_df_din_model['pred_score'] = tst_user_item_feats_df_din_model['pred_score'].transform(lambda x: norm_sim(x))
tst_user_item_feats_df_din_model.sort_values(by=['user_id', 'pred_score'])
tst_user_item_feats_df_din_model['pred_rank'] = tst_user_item_feats_df_din_model.groupby(['user_id'])['pred_score'].rank(ascending=False, method='first')

# Save the new features from the test set cross-validation
tst_user_item_feats_df_din_model[['user_id', 'click_article_id', 'pred_score', 'pred_rank']].to_csv(save_path + 'tst_din_cls_feats.csv', index=False)

In [ ]:
# Re-rank the prediction results and save the results.
rank_results = tst_user_item_feats_df_rank_model[['user_id', 'click_article_id', 'pred_score']]
rank_results['click_article_id'] = rank_results['click_article_id'].astype(int)
save_results(rank_results, topk=5, model_name='din')

## 7. Model Ensembling

### 7.1 Weighted Fusion

In [ ]:
# Read the ranking results files from multiple models.
lgb_ranker = pd.read_csv(save_path + 'lgb_ranker_score.csv')
lgb_cls = pd.read_csv(save_path + 'lgb_cls_score.csv')
din_ranker = pd.read_csv(save_path + 'din_rank_score.csv')

In [ ]:
rank_model = {'lgb_ranker': lgb_ranker,
              'lgb_cls': lgb_cls,
              'din_ranker': din_ranker}

In [ ]:
def get_ensumble_predict_topk(rank_model, topk=5):
    final_recall = rank_model['lgb_cls'].append(rank_model['din_ranker'])
    rank_model['lgb_ranker']['pred_score'] = rank_model['lgb_ranker']['pred_score'].transform(lambda x: norm_sim(x))

    final_recall = final_recall.append(rank_model['lgb_ranker'])
    final_recall = final_recall.groupby(['user_id', 'click_article_id'])['pred_score'].sum().reset_index()

    save_results(final_recall, topk=topk, model_name='ensemble_fuse')

In [ ]:
get_ensumble_predict_topk(rank_model)

### 7.2 Stacking

In [ ]:
# Read the result files generated from multiple models' cross-validation.
# Training set
trn_lgb_ranker_feats = pd.read_csv(save_path + 'trn_lgb_ranker_feats.csv')
trn_lgb_cls_feats = pd.read_csv(save_path + 'trn_lgb_cls_feats.csv')
trn_din_cls_feats = pd.read_csv(save_path + 'trn_din_cls_feats.csv')

# Test set
tst_lgb_ranker_feats = pd.read_csv(save_path + 'tst_lgb_ranker_feats.csv')
tst_lgb_cls_feats = pd.read_csv(save_path + 'tst_lgb_cls_feats.csv')
tst_din_cls_feats = pd.read_csv(save_path + 'tst_din_cls_feats.csv')

In [ ]:
# Concatenate the features from multiple model outputs.
finall_trn_ranker_feats = trn_lgb_ranker_feats[['user_id', 'click_article_id', 'label']]
finall_tst_ranker_feats = tst_lgb_ranker_feats[['user_id', 'click_article_id']]

for idx, trn_model in enumerate([trn_lgb_ranker_feats, trn_lgb_cls_feats, trn_din_cls_feats]):
    for feat in [ 'pred_score', 'pred_rank']:
        col_name = feat + '_' + str(idx)
        finall_trn_ranker_feats[col_name] = trn_model[feat]

for idx, tst_model in enumerate([tst_lgb_ranker_feats, tst_lgb_cls_feats, tst_din_cls_feats]):
    for feat in [ 'pred_score', 'pred_rank']:
        col_name = feat + '_' + str(idx)
        finall_tst_ranker_feats[col_name] = tst_model[feat]

In [ ]:
# Define a logistic regression model to fit the features generated from cross-validation and predict on the test set.
from sklearn.linear_model import LogisticRegression

feat_cols = ['pred_score_0', 'pred_rank_0', 'pred_score_1', 'pred_rank_1', 'pred_score_2', 'pred_rank_2']

trn_x = finall_trn_ranker_feats[feat_cols]
trn_y = finall_trn_ranker_feats['label']

tst_x = finall_tst_ranker_feats[feat_cols]

# Define the model
lr = LogisticRegression()

# Train the model
lr.fit(trn_x, trn_y)

# Predict the model
finall_tst_ranker_feats['pred_score'] = lr.predict_proba(tst_x)[:, 1]

In [ ]:
# Re-rank the prediction results and save the results.
rank_results = finall_tst_ranker_feats[['user_id', 'click_article_id', 'pred_score']]
save_results(rank_results, topk=5, model_name='ensumble_staking')

## 8. Summary
In this section, we implemented and evaluated three distinct ranking models for a recommendation system:

1. LightGBM Ranker: This is a LightGBM model specifically optimized for ranking. It learns to order items based on their relative importance, which is ideal for a recommendation task where the goal is to find the best articles to show at the top of a list.

2. LightGBM Classifier: This is a LightGBM model that predicts a binary outcome—whether a user will click on an article or not. The output of this model is a probability score that can be used to rank the articles from most likely to least likely to be clicked.

3. DIN (Deep Interest Network): This is a powerful deep learning model that uses an attention mechanism to dynamically capture a user's evolving interests. It's particularly effective for problems like news recommendations where a user's preferences change over time.

### Model Fusion 
By implementing these three models, we have a diverse set of predictions for our test data. By combining the outputs of these different models, for example, using model fusion or a simple stacking model with logistic regression, we achieve a final result that is more accurate and robust than any single model alone. This is because each model may capture different patterns and signals from the data.